## Part 3 - Train Model

**Connect to Azure ML workspace and get default datastore**

In [1]:
from azureml.core import Workspace

workspace = Workspace.from_config()
datastore = workspace.get_default_datastore()

**Retrieve dataset and compute targets prepared in earlier steps**

In [21]:
cifar_dataset = datastore.path('cifar')
compute_target = workspace.compute_targets['k80cluster']

**Create experiment**

In [22]:
from azureml.core import Experiment

experiment_name = 'objection-recognition' 
experiment = Experiment(workspace, name=experiment_name)

**Submit training job**

In [17]:
%pycat train/train.py

In [23]:
from azureml.train.estimator import Estimator

estimator = Estimator(source_directory='train',
                      entry_script='train.py',
                      compute_target=compute_target,
                      script_params = {
                          '--data_dir': cifar_dataset.as_mount(),
                          '--output_dir': './outputs', 
                          '--num_epochs': 10, 
                          '--batch_size': 16,
                          '--learning_rate': 0.001, 
                          '--momentum': 0.9
                      },
                      use_gpu=True,
                      pip_packages=['torch==1.4.0', 'torchvision==0.5.0'])

In [24]:
run = experiment.submit(estimator)

In [25]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

**Register model**

In [26]:
model = run.register_model(model_name='cifar-classifier', model_path='outputs')